In [1]:
import quaternion as q
import numpy as np
import pykitti
import matplotlib.pyplot as plt

In [80]:
directoryL = "/home/sexy/Documents/dataset/"
list_Y = [],[]
for i in range(11):
    data = pykitti.odometry(directoryL, str(i).zfill(2))
    data.load_poses()

    #Empty arrays to hold data
    Translations = np.zeros((len(data.T_w_cam0),3))
    Quaternions = np.zeros(len(data.T_w_cam0), dtype=q.quaternion)

    #store quaternions and translations
    for j in range(len(data.T_w_cam0)):
        Translations[j,:] = data.T_w_cam0[j][:-1,-1]
        Quaternions[j] = (q.from_rotation_matrix(data.T_w_cam0[j][:-1,:-1])).normalized()

    # Make empty arrays to hold data
    delta_trans = np.zeros((len(data.T_w_cam0),3))
    corrected_delta_trans = np.zeros((len(data.T_w_cam0),3))
    delta_quat = np.zeros(len(data.T_w_cam0),dtype=q.quaternion)

    #Get translations between frames
    delta_trans[:-1,:] = Translations[1:,:]-Translations[:-1,:]

    #Correct translations to provide relative motion from car's perspective
    for k in range(len(delta_trans)-1):
        corrected_delta_trans[k] = (Quaternions[k]*q.quaternion(0.,*delta_trans[k])*Quaternions[k].conj()).vec

    #get rotations between frames
    delta_quat[:-1] = Quaternions[:-1].conj()*Quaternions[1:]

    list_Y[0].append(corrected_delta_trans[:-1])
    list_Y[1].append(delta_quat[:-i])

Loading poses for sequence 00...
done.
Loading poses for sequence 01...
done.
Loading poses for sequence 02...
done.
Loading poses for sequence 03...
done.
Loading poses for sequence 04...
done.
Loading poses for sequence 05...
done.
Loading poses for sequence 06...
done.
Loading poses for sequence 07...
done.
Loading poses for sequence 08...
done.
Loading poses for sequence 09...
done.
Loading poses for sequence 10...
done.


In [82]:
print(list_Y[0][10].shape)

(1200, 3)
